In [ ]:
import os
import polars as pl
import numpy as np
import plotly.express as px

from datetime import datetime, timezone

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

In [ ]:
df_p_dwd = pl.read_parquet(os.path.join(DATA_DIRECTORY, "processed", "picarro", "Calibrated_Raw_DWD_Picarro_G2301_413.parquet"))

In [ ]:
def process_calibration_time(df, start_date, end_date):
    df_filtered = df.filter(pl.col("datetime").is_between(start_date, end_date))
    
    fig = px.line(df_filtered, x="datetime", y="picarro_corrected")
    fig.show()
    
    fig = px.line(df_filtered, x="datetime", y="h2o_reported")
    fig.show()
    
    data = df_filtered.select(pl.col("picarro_corrected")).to_series().to_list()
        
    return np.median(data[int(len(data)*0.3):int(len(data)*0.95)])

In [ ]:
# 81 12:44 - 13:04
# Reading: 518.9

start_date = datetime(2024, 12, 11, 12, 44, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 13, 4, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 82 13:08 - 13:29
# Reading: 518.3

start_date = datetime(2024, 12, 11, 13, 8, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 13, 28, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 83 15:25 - 15:45
# Reading: 518.9

start_date = datetime(2024, 12, 3, 15, 25, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 3, 15, 45, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 84  16:05 - 16:25
# Reading: 516.0

start_date = datetime(2024, 12, 3, 16, 5, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 3, 16, 25, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 85 UTC: 13:31 - 13:51
# Reading: 519.0

start_date = datetime(2024, 12, 11, 13, 31, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 13, 51, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 86 UTC: 13:53 - 14:13
# Reading: 518.7

start_date = datetime(2024, 12, 11, 13, 53, 0).replace(tzinfo=timezone.utc)
end_dhate = datetime(2024, 12, 11, 14, 13, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 87 UTC: 14:15 - 14:35
# Reading: 518.8

start_date = datetime(2024, 12, 11, 14, 15, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 14, 0, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 88 UTC: 14:38 - 14:58
# Reading: 518.4

start_date = datetime(2024, 12, 11, 14, 38, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 14, 58, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 89 UTC: 15:01 - 15:21
# Reading: 519.6

start_date = datetime(2024, 12, 11, 15, 1, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 15, 21, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

In [ ]:
# 90 UTC: 15: - 15:
# Reading: 518.3

start_date = datetime(2024, 12, 11, 15, 1, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 11, 15, 0, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)